# Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt

# Import data

In [3]:
# import stock data 

#define yesterday 

yesterday = dt.date.today()

last_5_years = dt.date.today() - dt.timedelta(days=1825)

tickers = ['IWFM.L',
'IWFV.L',
'CNDX.L',
'WQDS.L',
'EXI5.DE',
'XRES.L',
'SGIL.L',
'CMOP.L',
'ICOM.L',
'BTC-GBP']

# use a for loop to fetch the "Close" price data for each ticker
dfs = []
for ticker in tickers:
    t = yf.Ticker(ticker)
    data = t.history(start=last_5_years, end=yesterday)
    data = pd.DataFrame(data['Close'])
    data.columns = [ticker]
    data.index = data.index.strftime('%Y-%m-%d')
    dfs.append(data)

# Merge dataframes using index date
df = dfs[0]
for i in range(1, len(dfs)):
    df = pd.merge(df, dfs[i], how='outer', left_index=True, right_index=True)

# Rename the columns with the tickers
df.columns = tickers

df.head(5)

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP
Date,,,,,,,,,,
2018-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4919.691406
2018-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4978.958984
2018-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4876.196777
2018-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5043.645508
2018-04-03,2674.5,2230.5,357.390015,365.299988,17.594,14.85,114.904999,1263.0,5.28,5300.426270


In [4]:
# fill all empty values with data

df = df.dropna()

df.isnull().sum()

IWFM.L     0
IWFV.L     0
CNDX.L     0
WQDS.L     0
EXI5.DE    0
XRES.L     0
SGIL.L     0
CMOP.L     0
ICOM.L     0
BTC-GBP    0
dtype: int64

# Correct ticker currencies so that they are all in GBP 

In [5]:
# define ticker currency

ticker_currency = {'IWFM.L': 'GBP', 
               'IWFV.L': 'GBP', 
               'CNDX.L': 'USD', 
               'WQDS.L': 'GBP', 
               'EXI5.DE': 'EUR', 
               'XRES.L': 'USD', 
               'SGIL.L': 'GBP', 
               'CMOP.L': 'GBP', 
               'ICOM.L': 'USD', 
               'BTC-GBP': 'GBP'}

In [6]:
# Get GBP/USD and GBP/EUR exchange rates from Yahoo Finance
fx = yf.Ticker("USDGBP=X").history(start=last_5_years, end=yesterday)["Close"]
fx = pd.DataFrame(fx)
fx.columns = ['USDGBP']
fx.index = fx.index.strftime('%Y-%m-%d')
fx_eur = yf.Ticker("EURGBP=X").history(start=last_5_years, end=yesterday)["Close"]
fx_eur = pd.DataFrame(fx_eur)
fx_eur.columns = ['EURGBP']
fx_eur.index = fx_eur.index.strftime('%Y-%m-%d')

# Merge exchange rate data with original dataframe on date index
df = pd.merge(df, fx, left_index=True, right_index=True, how='left')
df = pd.merge(df, fx_eur, left_index=True, right_index=True, how='left')

df.tail(2)

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP,USDGBP,EURGBP
Date,,,,,,,,,,,,
2023-03-27,4324.0,2839.0,722.929993,475.350006,11.332,19.762501,125.160004,1805.75,6.5975,22861.289062,0.816593,0.87950
2023-03-28,4350.0,2839.5,714.494995,473.399994,11.094,19.642500,124.739998,1816.50,6.6900,22070.970703,0.813170,0.87874


In [7]:
# Convert non-GBP tickers to GBP
for ticker in tickers:
    if ticker_currency[ticker] != 'GBP':
        target_fx_rate = df['EURGBP'] if ticker_currency[ticker] == 'EUR' else df['USDGBP']
        df[ticker] = df[ticker] * target_fx_rate

df.tail(2)     

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP,USDGBP,EURGBP
Date,,,,,,,,,,,,
2023-03-27,4324.0,2839.0,590.339565,475.350006,9.966493,16.137920,125.160004,1805.75,5.387472,22861.289062,0.816593,0.87950
2023-03-28,4350.0,2839.5,581.005906,473.399994,9.748742,15.972692,124.739998,1816.50,5.440107,22070.970703,0.813170,0.87874


In [8]:
# convert GBX tickers to GBP 

df[['IWFM.L', 'IWFV.L', 'WQDS.L', 'CMOP.L']] = df[['IWFM.L', 'IWFV.L', 'WQDS.L', 'CMOP.L']]/100

df.tail(2)   

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP,USDGBP,EURGBP
Date,,,,,,,,,,,,
2023-03-27,43.24,28.390,590.339565,4.7535,9.966493,16.137920,125.160004,18.0575,5.387472,22861.289062,0.816593,0.87950
2023-03-28,43.50,28.395,581.005906,4.7340,9.748742,15.972692,124.739998,18.1650,5.440107,22070.970703,0.813170,0.87874


In [9]:
# drop currency columns 

# Remove column name 'A'
df = df.drop(['USDGBP', 'EURGBP'], axis=1)

df.tail(2)

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP
Date,,,,,,,,,,
2023-03-27,43.24,28.390,590.339565,4.7535,9.966493,16.137920,125.160004,18.0575,5.387472,22861.289062
2023-03-28,43.50,28.395,581.005906,4.7340,9.748742,15.972692,124.739998,18.1650,5.440107,22070.970703


# Implement PyPortfolioOpt

In [10]:
# follow https://www.youtube.com/watch?v=9fjs8FeLMJk

returns = df.pct_change()

returns.head(5)

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP
Date,,,,,,,,,,
2018-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-04-04,-0.004113,0.000224,-0.001001,-0.000821,-0.004057,-0.001559,-0.001523,-0.010689,-0.011014,-0.082080
2018-04-05,0.033790,0.021291,0.026939,0.024110,0.010198,0.008036,-0.000436,0.013205,0.004350,-0.000363
2018-04-06,-0.022880,-0.014483,-0.008968,-0.012172,0.004446,0.007676,-0.001918,-0.008886,0.003752,-0.031660
2018-04-09,0.002230,0.002227,-0.000507,0.000000,-0.003806,-0.008531,-0.002534,0.008767,0.005478,0.017366


In [11]:
# create annualized covariance matrix 

cov_matrix_annual = returns.cov() * 252 

cov_matrix_annual

,IWFM.L,IWFV.L,CNDX.L,WQDS.L,EXI5.DE,XRES.L,SGIL.L,CMOP.L,ICOM.L,BTC-GBP
IWFM.L,0.033376,0.023045,0.032928,0.019590,0.015497,0.019494,0.000978,0.008863,0.008802,0.027420
IWFV.L,0.023045,0.030138,0.025955,0.022590,0.019708,0.020476,-0.000453,0.008490,0.010278,0.022369
CNDX.L,0.032928,0.025955,0.060352,0.021170,0.029408,0.034883,-0.002963,0.003823,0.018120,0.034997
WQDS.L,0.019590,0.022590,0.021170,0.021442,0.015312,0.019412,0.000589,0.007193,0.007198,0.015432
EXI5.DE,0.015497,0.019708,0.029408,0.015312,0.055286,0.031677,-0.000702,0.001982,0.010879,0.029640
XRES.L,0.019494,0.020476,0.034883,0.019412,0.031677,0.053448,-0.001107,0.003321,0.016074,0.015102
SGIL.L,0.000978,-0.000453,-0.002963,0.000589,-0.000702,-0.001107,0.009786,0.003248,-0.001252,0.002221
CMOP.L,0.008863,0.008490,0.003823,0.007193,0.001982,0.003321,0.003248,0.026350,0.021931,0.013917
ICOM.L,0.008802,0.010278,0.018120,0.007198,0.010879,0.016074,-0.001252,0.021931,0.033723,0.018449
BTC-GBP,0.027420,0.022369,0.034997,0.015432,0.029640,0.015102,0.002221,0.013917,0.018449,0.484086


Variance measures how much each stock returns differ from each other

In [12]:
# Calculate the portfolio variance 

# assign weight to each stock 

weights = np.array([0.05, 0.05, 0.1, 0.2, 0.025, 0.025, 0.2, 0.15, 0.15, 0.05])

port_variance = np.dot(weights.T, np.dot(cov_matrix_annual, weights))

port_variance

0.012819080040801087

In [13]:
# calculate portfolio volatility and standard deviation 

port_volatility = np.sqrt(port_variance)

port_volatility

0.11322137625378471

In [14]:
# calculate the annual portfolio return 

portfolioSimpleAnnualReturn = np.sum(returns.mean() * weights) * 252

portfolioSimpleAnnualReturn

0.10032709515328335

In [15]:
# expected annual return, volatility (aka risk) and variance 

percent_var = str( round(port_variance, 2)* 100)+'%'

percent_vol = str( round(port_volatility, 2)* 100)+'%'

percent_ret = str( round(portfolioSimpleAnnualReturn, 2)* 100)+'%'

print("Expected annual return: "+ percent_ret)

print("Expected volatility or risk: " + percent_vol)

print("Annual variance: "+ percent_var)


Expected annual return: 10.0%
Expected volatility or risk: 11.0%
Annual variance: 1.0%


In [16]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [17]:
# run portfolio optimisation 

# calcualte expected returns and annualized sample covariance matrix 

mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

#optimize for max sharpe ratio

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('IWFM.L', 0.0), ('IWFV.L', 0.0), ('CNDX.L', 0.53737), ('WQDS.L', 0.0), ('EXI5.DE', 0.0), ('XRES.L', 0.0), ('SGIL.L', 0.0), ('CMOP.L', 0.36195), ('ICOM.L', 0.0), ('BTC-GBP', 0.10068)])
Expected annual return: 15.9%
Annual volatility: 17.9%
Sharpe Ratio: 0.78


(0.15922493725318104, 0.1790934630630686, 0.7773870406657576)

In [22]:
# get the discrete allocation of each share per stock 

from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)

weights = cleaned_weights

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=10000)

allocation, leftover = da.lp_portfolio()

print("Discrete allocation: ", allocation)
print("Funds remaining: ${:.2f}".format(leftover))



Discrete allocation:  {'CNDX.L': 9, 'CMOP.L': 199}
Funds remaining: $1156.11
